In [2]:
# Nika's notebook

# Import statements
import numpy as np
import pandas as pd
from pathlib import Path
%matplotlib inline
import datashader as ds

In [163]:
# read the uscities csv. this file will be used for the longitude and latitude, as well as the density stats.
longlatdensity_csv = Path("../Data/uscities.csv")
longlatdensity_df = pd.read_csv(longlatdensity_csv)
longlatdensity_df.head(5)

#getting first zipcode from list of zipcodes
longlatdensity_df[['firstzip']] = longlatdensity_df['zips'].str.split(",", n = 1, expand=True)
longlatdensity_df[['firstzip']] = longlatdensity_df['zips'].str[:5]

#dropping unnecessary columns
longlatdensity_df.drop(columns=['source','military','incorporated','timezone','city_ascii',"zips","id","county_fips","county_name"],inplace=True)
longlatdensity_df.drop_duplicates()
longlatdensity_df=longlatdensity_df.rename(columns={"city": "Primary MSA"})
longlatdensity_df=longlatdensity_df.set_index("Primary MSA")
longlatdensity_df

,state_id,state_name,lat,lng,population,density,ranking,firstzip
Primary MSA,,,,,,,,
New York,NY,New York,40.6943,-73.9249,18713220.0,10715.0,1,11229
Los Angeles,CA,California,34.1139,-118.4068,12750807.0,3276.0,1,90291
Chicago,IL,Illinois,41.8373,-87.6862,8604203.0,4574.0,1,60018
Miami,FL,Florida,25.7839,-80.2102,6445545.0,5019.0,1,33129
Dallas,TX,Texas,32.7936,-96.7662,5743938.0,1526.0,1,75287
...,...,...,...,...,...,...,...,...
Caputa,SD,South Dakota,43.9960,-102.9847,0.0,0.0,3,57703
Hamill,SD,South Dakota,43.5939,-99.6907,0.0,0.0,3,57534
Loomis,SD,South Dakota,43.7931,-98.1036,0.0,0.0,3,57301


In [165]:
# Using median income CSV files from EH notebook. 
# data cleanup using EH notebook. Differences are: (created new medican_income_affdf for affordability df calc)

median_income_csv = Path("../Data/household_income_msa_2019.csv")
median_income_df = pd.read_csv(median_income_csv)
median_income_df = median_income_df.transpose() # Transpose the column/headers
new_header = median_income_df.iloc[0] #grab the first row for the header
median_income_df = median_income_df[1:] #take the data less the header row
median_income_df.columns = new_header #set the header row as the df header
median_income_df = median_income_df[['Median income (dollars)','Mean income (dollars)']] # Use only median and mean income data columns
median_income_df.reset_index(inplace=True)
median_income_df[['MSA','extra']] = median_income_df['index'].str.split(",", n = 1, expand=True) # Split MSA from text string
median_income_df[['State','extra2']] = median_income_df['extra'].str.split(n = 1, expand=True) # Split State from remaining text string
median_income_df[['Primary MSA','Secondary MSA']] = median_income_df['MSA'].str.split("-", n = 1, expand=True) # Split Primary MSA from first text string
median_income_df[['Primary State','Secondary State']] = median_income_df['State'].str.split("-", n = 1, expand=True) # Split Primary MSA from first text string
median_income_df.drop(columns=['index','extra','extra2','MSA','State'],inplace=True) # Drop unnecessary columns
median_income_df.drop_duplicates(subset=['Primary MSA','Primary State'], inplace=True) # Drop duplicates -- first instance is the median and mean income by MSA
median_income_df.head()

#creating filtered dataframe for affordability calc

median_income_affdf=median_income_df.filter(['Primary MSA','Median income_(dollars)','Mean income (dollars)'], axis=1)
median_income_affdf=median_income_affdf.set_index("Primary MSA")
median_income_affdf


Label,Mean income (dollars)
Primary MSA,
Aberdeen,"77,948"
Abilene,"71,517"
Adrian,"69,443"
Aguadilla,"25,781"
Akron,"80,800"
...,...
York,"85,972"
Youngstown,"64,409"
Yuba City,"82,466"


In [201]:
# Getting 2019 rental prices to compare to the 2019 median income prices from above

#Data cleanup using AH notebook. 
#Differences are: (1.used the same EH primary MSA split from City,  2. did not transpose data, 3. created newdf for affordability calc)

csv_path_2 = Path("../Data/rental_prices_msa.csv")
rental_price_2019 = pd.read_csv(csv_path_2, usecols=['RegionName','2019-01',
 '2019-02',
 '2019-03',
 '2019-04',
 '2019-05',
 '2019-06',
 '2019-07',
 '2019-08',
 '2019-09',
 '2019-10',
 '2019-11',
 '2019-12',]) # used rental_price_raw.columns.tolist() to get the list of 2019 columns


#splitting City and State
rental_price_2019[['City','State']] = rental_price_2019.RegionName.str.split(",",expand=True)
rental_price_2019[['Primary MSA','Secondary MSA']] = rental_price_2019['City'].str.split("-",n=1,expand=True)
city_col_name="Primary MSA"
city2_col_name="Secondary MSA"
state_col_name="State"
region_col_name="RegionName"
first_col = rental_price_2019.pop(city_col_name)
second_col = rental_price_2019.pop(city2_col_name)
third_col = rental_price_2019.pop(state_col_name)
fourth_col = rental_price_2019.pop(region_col_name)
rental_price_2019.insert(0, city_col_name, first_col)
rental_price_2019.insert(1, city2_col_name,second_col)
rental_price_2019.insert(1, state_col_name, third_col)
rental_price_2019.insert(2, region_col_name, fourth_col)
rental_price_2019.drop(["RegionName","City"], axis = 1, inplace = True)
rental_price_2019

#Getting annual rent
rental_price_2019['Annual_Rent']= rental_price_2019.iloc[:, 1:].sum(axis=1)
rental_price_2019

pd.set_option('display.max_rows', None)

#creating new to use for affordability calc
rental_price_2019_affdf=rental_price_2019.filter(['Primary MSA','Annual_Rent'], axis=1)
rental_price_2019_affdf=rental_price_2019_affdf.set_index("Primary MSA")
rental_price_2019_affdf


,Annual_Rent
Primary MSA,
United States,20106
New York,32189
Los Angeles,30186
Chicago,21155
Dallas,18506
Philadelphia,18942
Houston,17720
Washington,25314
Miami,22838


In [202]:
#Concatenate 3 datasets (uscities, rentalprice, median income)

firstmerge = pd.merge(longlatdensity_df, median_income_affdf, on="Primary MSA")
affordability_df= pd.merge(firstmerge, rental_price_2019_affdf, on="Primary MSA")
affordability_df

,state_id,state_name,lat,lng,population,density,ranking,firstzip,Mean income (dollars),Annual_Rent
Primary MSA,,,,,,,,,,
New York,NY,New York,40.6943,-73.9249,18713220.0,10715.0,1,11229,"122,967",32189
Los Angeles,CA,California,34.1139,-118.4068,12750807.0,3276.0,1,90291,"109,583",30186
Los Angeles,TX,Texas,26.4947,-97.7863,0.0,0.0,3,78580,"109,583",30186
Chicago,IL,Illinois,41.8373,-87.6862,8604203.0,4574.0,1,60018,"104,357",21155
Miami,FL,Florida,25.7839,-80.2102,6445545.0,5019.0,1,33129,"89,694",22838
Miami,OK,Oklahoma,36.8877,-94.8718,15571.0,466.0,3,74354,"89,694",22838
Miami,AZ,Arizona,33.3951,-110.8721,1780.0,783.0,3,85539,"89,694",22838
Miami,TX,Texas,35.6923,-100.6416,550.0,181.0,3,79059,"89,694",22838
Miami,MO,Missouri,39.3224,-93.2258,169.0,116.0,3,65344,"89,694",22838


In [198]:
#working on this now -- there are multiple cities named WINSTON in different states in the long lat csv file! 

rental_price_2019_df2=rental_price_2019.reset_index()

checkrental_price_2019=rental_price_2019_df2[rental_price_2019_df2['Primary MSA'].str.contains('Winston')]
checkrental_price_2019

#affordability_df2=affordability_df.reset_index()

#checkaffordabilitydf=affordability_df2[affordability_df2['Primary MSA'].str.contains('Winston')]
#checkaffordabilitydf

longlatdensity_df2=longlatdensity_df.reset_index()
checklonglat=longlatdensity_df2[longlatdensity_df2['Primary MSA'].str.contains('-')]
checklonglat

,Primary MSA,state_id,state_name,lat,lng,population,density,ranking,firstzip
102,Winston-Salem,NC,North Carolina,36.1029,-80.2611,422269.0,721.0,2,27110
457,Arden-Arcade,CA,California,38.6034,-121.3810,100548.0,2184.0,2,95864
767,Florence-Graham,CA,California,33.9682,-118.2447,64162.0,7059.0,2,90001
1209,Wilkes-Barre,PA,Pennsylvania,41.2468,-75.8759,40766.0,2254.0,2,18702
1296,Security-Widefield,CO,Colorado,38.7489,-104.7142,38134.0,1132.0,3,80911
...,...,...,...,...,...,...,...,...,...
28656,Morales-Sanchez,TX,Texas,26.7876,-99.1147,38.0,10.0,3,78076
29105,Valley-Hi,PA,Pennsylvania,40.0299,-78.1917,15.0,11.0,3,15533
29204,Elias-Fela Solis,TX,Texas,26.3855,-98.6238,8.0,13.0,3,78582
29371,Pope-Vannoy Landing,AK,Alaska,59.5649,-154.4654,0.0,0.0,3,99613
